# Evaluating definite integrals using `TaylorIntegration.jl`

The aim of this example is to show that using `TaylorIntegration.jl` we can evaluate with high accuracy definite integrals of the form 

$$
I_{\omega}(x) = \int_{x_0}^xf_{\omega}(t)\mathrm{d} t
$$

for smooth integrands $f$ which might depend on a parameter $\omega$. In particular, combining high-acurracy integration with jet transport techniques, we are able to obtain high-order, dense polynomial representations of special functions and integral transforms. The trick is to translate the definite integral problem to an initial value problem for an explicit ODE, thanks to the fundamental theorem of calculus.

## 1. Error function: $\operatorname{erf}(x)=\frac{2}{\sqrt{\pi}}\int_0^x \exp(-t^2) dt$

As a first example, we will evaluate the [__error function__](https://en.wikipedia.org/wiki/Error_function), defined by the integral $\operatorname{erf}(x)=\frac{2}{\sqrt{\pi}}\int_0^x \exp(-t^2) dt$ using `TaylorIntegration.jl`. We proceed as follows:

First, define a suitable ODE system equivalent to the definite integral problem at hand, and then integrate it. Indeed, by the fundamental theorem of calculus we have

$$
\operatorname{erf} '(x) =\frac{2}{\sqrt{\pi}}\exp(-x^2)
$$


where the prime $'$ denotes differentiation with respect to $x$. Therefore, we reinterpret $\operatorname{erf}(x)$ as the solution of the initial value problem defined by the ODE

$$
\frac{d\operatorname{erf}}{dx} = \frac{2}{\sqrt{\pi}}\exp(-x^2)
$$

subject to the initial condition $\operatorname{erf}(0)=0$. We will integrate this ODE in order to evaluate $\operatorname{erf}(x)$.

We star by loading `TaylorIntegration` and `Plots` with the `GR` backend:

In [7]:
using TaylorIntegration, Plots
gr()

Plots.GRBackend()

This is the RHS of the ODE we want to solve (below, `t` is the independent variable):

In [8]:
f(t, x) = (2/sqrt(pi))*exp(-t^2)

f (generic function with 1 method)

The parameters we will use for the Taylor integration are:

In [9]:
x0 = 0.0 #the initial value of the independent variable; in this case, x
xmax = 10.0 #the final value of the independent variable; in this case, x
erf0 = 0.0 #the initial condition for erf: erf(0) = 0
const order = 25 #the order of the Taylor expansions
const abstol = 1e-20 #the absolute local error tolerance

1.0e-20

The initial conditions are:

In [10]:
x0, erf0

(0.0, 0.0)

Then, we proceed to integrate:

In [11]:
@time xv, erfv = taylorinteg(f, erf0, x0, xmax, order, abstol, maxsteps=5000);

  0.354741 seconds (147.51 k allocations: 7.759 MiB, 2.77% gc time)


The final $x$ and $\operatorname{erf}$ values are:

In [12]:
xv[end], erfv[end]

(10.0, 1.0)

Now, how does $\operatorname{erf}(x)$ look as a function of $x$?

In [14]:
plot(xv, erfv, leg=false)
scatter!(xv, erfv, ms=3.0)

xlabel!("x")
ylabel!("erf(x)")
title!("Evaluating erf(x) using TaylorIntegration")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 0.00 
 
 
 0.25 
 
 
 0.50 
 
 
 0.75 
 
 
 1.00 
 
 
 Evaluating erf(x) using TaylorIntegration 
 
 
 x 
 
 
 erf(x)

At a first glance, this looks great! But, how do these values actually compare to `SpecialFunctions.jl`'s `erf` implementation?

In [15]:
import SpecialFunctions: erf

In [16]:
?Base.erf

```
erf(x)
```

Compute the error function of `x`, defined by $\frac{2}{\sqrt{\pi}} \int_0^x e^{-t^2} dt$ for arbitrary complex `x`.


Let's plot the absolute difference between our integrated values for $\operatorname{erf}(x)$ and `Base.erf`, in units of the machine epsilon, as a function of $x$:

In [17]:
plot(
    xv, (erfv-erf.(xv))/eps(),
    xaxis = "x",
    yaxis = "Diff: integrated vs Base.erf",
    title = "Absolute difference between erf(x) and I(x)",
    leg = false
)

scatter!(xv, (erfv-erf.(xv))/eps(), ms=3.0)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 0.0 
 
 
 0.1 
 
 
 0.2 
 
 
 0.3 
 
 
 0.4 
 
 
 0.5 
 
 
 Absolute difference between erf(x) and I(x) 
 
 
 x 
 
 
 Diff: integrated vs Base.erf

So the error is at most half times `eps()` at each time step!!!!

## 2. High-accuracy evaluation of elliptic integrals with `BigFloats`

In this section, we will evaluate the elliptic integral $K$ using `BigFloats`, up to an accuracy of about $10^{-77}$ (yes, `1E-77`!). It is well known that the complete elliptic integral of the first kind, $K(k)$, may be evaluated via the integral

$$
K(k) = \int_0^{\pi/2}\frac{ \mathrm{d}t }{ \sqrt{1-k^2\sin^2 t} }
$$

Again, we can evaluate this integral using `TaylorIntegration.jl`, if we reinterpret this definite integral as the solution of the ODE defined by

$$
\frac{\mathrm{d}K_k}{\mathrm{d}t} = \frac{ 1 }{ \sqrt{1-k^2\sin^2 t} }
$$

integrated from $t=0$ to $t=\pi/2$ for a given parameter $k$, subject to the initial condition $K_k(t=0)=0$.

For definiteness, we will select the parameter value $k_0=0.6051864057360395$.

In [103]:
k0 = 0.6051864057360395
k = big"0.6051864057360395"

6.051864057360394999999999999999999999999999999999999999999999999999999999999984e-01

Then, $k^2$ is:

In [104]:
k^2

3.662505856877062234277491455602499999999999999999999999999999999999999999999966e-01

The RHS of the obtained ODE may be written as:

In [105]:
G(t,x) = 1/sqrt(1-(k*sin(t))^2)

G (generic function with 1 method)

Since we will evaluate this integral using  `BigFloat`s, we will select an order of 90, and a local absolute tolerance near `eps(BigFloat)`:

In [106]:
const bigorder = 90
const bigabstol = big"1.0E-77"

1.000000000000000000000000000000000000000000000000000000000000000000000000000004e-77

Now, we perform the integration:

In [107]:
@time tv, xv = taylorinteg(G, 0.0, 0.0, BigFloat(π)/2, bigorder, bigabstol);

  2.172711 seconds (18.34 M allocations: 912.970 MiB, 45.24% gc time)


The value of $K(k)$ corresponds to the last value of `xv[end]`:

In [108]:
xv[end]

1.754812577961136589734955441226555912336722052661065250219997312395371169887374

Using `Elliptic.jl`, we can obtain the first few digits of the value of $K(k)$, for the selected value of $k$:

In [109]:
import Elliptic: K

In [110]:
K(k^2)

1.7548125779611365

Furthermore, using [`ArbFloats.jl`](https://github.com/JuliaArbTypes/ArbFloats.jl), we can obtain an extended precision evaluation of the elliptic integral $K$ (thanks to [@JeffreySarnoff](https://github.com/JeffreySarnoff) for this!):

In [111]:
using ArbFloats

arb_precision = 270 # bits, select: 100..3200
setprecision(ArbFloat, arb_precision)

function ellipticK(x::ArbFloat)
    arb_one    = one(ArbFloat)
    arb_halfpi = ArbFloat(pi)/2
        
    k = sqrt( abs(x - arb_one) )
    k = agm(arb_one, k)
    k = arb_halfpi / k

    return k
end

ellipticK (generic function with 2 methods)

In [112]:
showall(  ellipticK( (@ArbFloat 0.6051864057360395)^2 )  )

1.754812577961136589734955441226555912336722052661065250219997312395371169887377

Which differs from the result obtained by `TaylorIntegration.jl` __*only at the last digit !!!*__ Thus, `TaylorIntegration.jl` is able to evaluate special functions with *really* high accuracy.

## 3. High-order polynomial approximations to special functions

Now, what would happen if we wanted to evaluate the elliptic integral $K(k)$, not for one particular value, but for several values of the parameter $k$? Should we evaluate them one by one? If the set of values of $k$ lie within a small neighborhood of a given value of $k$, say $k_0$, then an alternative is to think of each of $k$ as a Taylor polynomial in $t$ ($t$ is a dummy variable), such that $k=k_0+t$. Using this idea, we can combine `TaylorSeries.jl` and `TaylorIntegration.jl` in order to obtain a high-order polynomial approximation to the special function $K$, around a given value $k_0$.

In [113]:
using TaylorSeries

In [114]:
t = Taylor1(15) # the dummy variable k around which we are expanding the parameter k

 1.0 t + 𝒪(t¹⁶)

In [125]:
k = k0 + t # re-thinking k as a Taylor polynomial in the dummy variable t

 0.6051864057360395 + 1.0 t + 𝒪(t¹⁶)

We will now integrate an ODE system where the first equation corresponds to the original problem of evaluating the elliptic integral $K$, whereas the second equation corresponds to the of the parameter $k$, but since the parameter $k$ is constant, we have $\dot k = 0$:

In [119]:
function G!(t, x, dx)
    dx[1] = 1/sqrt(1-(x[2]*sin(t))^2)
    dx[2] = zero(x[2])
    nothing
end

G! (generic function with 1 method)

The initial conditions for this new system are:

In [124]:
x0 = [zero(t), k]

2-element Array{TaylorSeries.Taylor1{Float64},1}:
                         0.0 + 𝒪(t¹⁶)
  0.6051864057360395 + 1.0 t + 𝒪(t¹⁶)

Now, we can integrate the system above:

In [126]:
@time tv, xv = taylorinteg(G!, x0, 0.0, π/2, 28, 1E-20);

  1.118427 seconds (2.45 M allocations: 185.524 MiB, 5.00% gc time)


Now, `xv[end,1]` corresponds to a 15-th order Taylor expansion around $k_0$ of the complete elliptic integral of the first kind, $K$:

In [130]:
xv[end,1]

 1.7548125779611359 + 0.7902217547288612 t + 1.4862006794969997 t² + 2.4723818499390178 t³ + 4.877283348455082 t⁴ + 9.96559188808282 t⁵ + 21.224636460154077 t⁶ + 46.32399841153163 t⁷ + 103.09191906811044 t⁸ + 232.82644779824827 t⁹ + 532.0565841031579 t¹⁰ + 1227.5568728333137 t¹¹ + 2854.8206184572873 t¹² + 6683.860182924387 t¹³ + 15738.519788056736 t¹⁴ + 37243.53671901891 t¹⁵ + 𝒪(t¹⁶)

We can exploit the function-like behavior (i.e., the "callability") of `Taylor1` variables in order to call this Taylor expansion of $K$ as if it were a function:

In [140]:
ellipticK_taylor(x) = xv[end,1](x)

ellipticK_taylor (generic function with 1 method)

Since we're expanding around $k_0$, note that `ellipticK_taylor(x) ≈ Elliptic.K((x+k0)^2)`:

In [145]:
ellipticK_taylor(0.0)-Elliptic.K(k0^2)

-6.661338147750939e-16

In [159]:
k_radius = 0.35
x = k0-k_radius:0.025:k0+k_radius

0.25518640573603957:0.025:0.9301864057360396

Plot `ellipticK_taylor` and `Elliptic.K` for $k \in [k_0-0.35,k_0+0.35]$:

In [167]:
plot(x, ellipticK_taylor.(x-k0), lab="Taylor approx")
plot!(x, Elliptic.K.(x.^2), lab="Elliptic.K")
xlabel!("k")
ylabel!("Elliptic integral K value")
title!("Taylor polynomial approximation vs Elliptic.K")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.3 
 
 
 0.4 
 
 
 0.5 
 
 
 0.6 
 
 
 0.7 
 
 
 0.8 
 
 
 0.9 
 
 
 1.6 
 
 
 1.8 
 
 
 2.0 
 
 
 2.2 
 
 
 2.4 
 
 
 Taylor polynomial approximation vs Elliptic.K 
 
 
 k 
 
 
 Elliptic integral K value 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Taylor approx 
 
 
 
 Elliptic.K

Now, let's plot the absolute difference between `ellipticK_taylor` and `Elliptic.K` for $k$ in the same interval:

In [174]:
plot(x, log10.(abs.(ellipticK_taylor.(x-k0)-Elliptic.K.(x.^2))), lab="log10(abs(diff))")
xlabel!("k")
ylabel!("abs difference (log10)")
title!("K: Taylor approximation vs Elliptic.K")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.3 
 
 
 0.4 
 
 
 0.5 
 
 
 0.6 
 
 
 0.7 
 
 
 0.8 
 
 
 0.9 
 
 
 -14 
 
 
 -12 
 
 
 -10 
 
 
 -8 
 
 
 -6 
 
 
 -4 
 
 
 K: Taylor approximation vs Elliptic.K 
 
 
 k 
 
 
 abs difference (log10) 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 log10(abs(diff))

So we see that near $k_0$ the difference between the Taylor polynomial approximation and `Elliptic.K` is below $10^{-14}$; and at the bounds of the interval the difference grows to $10^{-3}$. We can obtain better approximations using higher-order Taylor polynomials, up to a point where `Float64` can no longer give better accuracies, and then we would need to use either `BigFloat`s or `ArbFloat`s together with `TaylorSeries.jl` and `TaylorIntegration.jl`, which is readily possible! (exercise left to the reader...)

Furthermore, it is practically as efficient (if not even better!) to evaluate the elliptic integral $K$ with `Elliptic.jl` than it is to evaluate the Taylor polynomial approximation for it that we have obtained above:

In [175]:
using BenchmarkTools

In [176]:
k1 = k0+0.1

0.7051864057360395

In [177]:
@btime Elliptic.K(k1)

  267.396 ns (1 allocation: 16 bytes)


2.0830795049827855

In [178]:
@btime ellipticK_taylor(0.1)

  117.632 ns (1 allocation: 16 bytes)


1.8517837163136597

If one is dealing with some more complicated or custom-defined special functions, then this approach proves to be really useful!